In [ ]:
# импорт зависимостей
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# настройка matplotlib
%config InlineBackend.figure_format = "retina"
plt.style.use("seaborn-v0_8-notebook")

# путь к данным
DATA_DIR = Path("../data")

##### **Image padding**

In [ ]:
src_img = cv2.imread(DATA_DIR / "kitty.png", cv2.IMREAD_GRAYSCALE)

# define padding widths
left_pad = 20
top_pad = 40
right_pad = 80
bottom_pad = 160

# zero padding 
zero_padded_img = cv2.copyMakeBorder(
    src_img, 
    top_pad, bottom_pad, left_pad, right_pad, 
    cv2.BORDER_CONSTANT, 
    value=0
)

# replicate padding 
replicate_padded_image = cv2.copyMakeBorder(
    src_img, 
    top_pad, bottom_pad, left_pad, right_pad, 
    cv2.BORDER_REPLICATE
)

# reflect padding
reflect_padded_img = cv2.copyMakeBorder(
    src_img, 
    top_pad, bottom_pad, left_pad, right_pad, 
    cv2.BORDER_REFLECT
)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(6, 6))

ax1.imshow(src_img, cmap="gray")
ax1.axis("off")
ax1.set_title("original image")

ax2.imshow(zero_padded_img, cmap="gray")
ax2.axis("off")
ax2.set_title("zero padding")

ax3.imshow(replicate_padded_image, cmap="gray")
ax3.axis("off")
ax3.set_title("replicate padding")

ax4.imshow(reflect_padded_img, cmap="gray")
ax4.axis("off")
ax4.set_title("reflect padding")

plt.tight_layout()

##### **Свертка ядра с изображением**

In [ ]:
src_img = cv2.imread(DATA_DIR / "kitty.png", cv2.IMREAD_COLOR_RGB)

# размер ядра - нечетное число
n = 21

# зададим ядро n x n из единиц
w = np.ones(n)

# нормируем веса ядра, чтобы их сумма равнялась единице 
# это нужно для uint8 изображений
w = w / w.sum()

# свертка ядра w c изображением
dst_img = cv2.filter2D(
    src_img, 
    -1, 
    kernel=w, 
    borderType=cv2.BORDER_REFLECT
)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 18))

ax1.imshow(src_img)
ax1.axis("off")

ax2.imshow(dst_img)
ax2.axis("off")

plt.tight_layout()

##### **Корреляция ядра с изображением**

In [ ]:
src_img = cv2.imread(DATA_DIR / "kitty.png", cv2.IMREAD_COLOR_RGB)

# зададим ядро
w = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1],
])

# поскольку сумма весов равно нулю, нормировать ядро не надо

# свертка ядра w c изображением
dst_img = cv2.filter2D(
    src_img, 
    -1, 
    kernel=w.T, 
    borderType=cv2.BORDER_REFLECT
)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 18))

ax1.imshow(src_img)
ax1.axis("off")

ax2.imshow(dst_img)
ax2.axis("off")

plt.tight_layout()

##### **Свертка цветного изображения**

In [ ]:
src_img = cv2.imread(DATA_DIR / "cube.png", cv2.IMREAD_COLOR_RGB)

# ядро свертки
w = np.zeros((15, 15))

w[0, 0] = 0.5
w[-1, -1] = 0.5

# свертка ядра w c изображением
dst_img = cv2.filter2D(src_img, -1, kernel=w)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 18))

ax1.imshow(src_img)
ax1.axis("off")

ax2.imshow(dst_img)
ax2.axis("off")

plt.tight_layout()

##### **Сепарабельные свертки**

In [ ]:
src_img = cv2.imread(DATA_DIR / "kitty.png", cv2.IMREAD_COLOR_RGB)

# сепарабельное 2d ядро
w = np.array([
    [1, 2, 1],
    [2, 4, 2],
    [1, 2, 1]
])

# нормировка
w = w / w.sum()

# вектор u
u = np.array([[1, 2, 1]])
u = u / u.sum()

# вектор v
v = np.array([[1, 2, 1]])
v = v / v.sum()

# свертка с двумерным ядром w
dst_img_1 = cv2.filter2D(src_img, -1, w)

# сепарабельная свертка с одномерными ядрами u, v
dst_img_2 = cv2.sepFilter2D(src_img, -1, u, v)

# ошибка вычислений свертки двумя способами
errors = np.abs(dst_img_2.astype(int) - dst_img_1.astype(int))
print(f"max error = {errors.max()}")

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 18))

ax1.imshow(src_img)
ax1.axis("off")

ax2.imshow(dst_img_2)
ax2.axis("off")

plt.tight_layout()

##### **Box фильтр**

In [ ]:
# исходное изображение 
src_image = cv2.imread(DATA_DIR / "lena.png", cv2.IMREAD_COLOR_RGB)

# box-фильтрация 7 x 7

# w = np.ones(7)
# w = w / w.sum()
# dst_img = cv2.filter2D(src_img, -1, kernel=w)

dst_image = cv2.boxFilter(src_image, -1, ksize=(7, 7))

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 18))

ax1.imshow(src_image)
ax1.axis("off")

ax2.imshow(dst_image)
ax2.axis("off")

plt.tight_layout()

##### **Гауссовская фильтрация**

##### *Способ 1*

In [ ]:
# Построим гауссовское ядро
n = 65
k = n // 2

# одномерное гауссовское ядро
u = cv2.getGaussianKernel(ksize=n, sigma=0)

# двумерное сепарабельное гауссовское ядро
w = u @ u.T

# visualization
s, t = np.mgrid[-k:k:n*1j, -k:k:n*1j]

plt.figure(figsize=(10, 4))

plt.subplot(121)
plt.grid(True)
plt.plot(np.arange(-k, k + 1), u)

plt.subplot(122)
plt.title(f"n = {n}")
plt.axis("equal")
plt.pcolormesh(s, t, w)

plt.tight_layout()

In [ ]:
# исходное изображение 
src_image = cv2.imread(DATA_DIR / "rosso.png", cv2.IMREAD_COLOR_RGB)

n = 15

# гауссовская фильтрация 15 x 15, способ 1
u = cv2.getGaussianKernel(ksize=n, sigma=0)
w = u @ u.T

dst_image = cv2.filter2D(src_image, -1, kernel=w)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 18))

ax1.imshow(src_image)
ax1.axis("off")

ax2.imshow(dst_image)
ax2.axis("off")

plt.tight_layout()


##### *Способ 2*

In [ ]:
# гауссовская фильтрация 15 x 15
n = 15
dst_image = cv2.GaussianBlur(src_image, ksize=(n, n), sigmaX=0)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 18))

ax1.imshow(src_image)
ax1.axis("off")

ax2.imshow(dst_image)
ax2.axis("off")

plt.tight_layout()

##### **Медианная фильтрация**

In [ ]:
def add_salt_and_pepper_noise(src_image, noise_level=0.1):
    noised_image = src_image.copy()
    image_size = noised_image.size

    noise_size = int(noise_level * image_size)
    random_indices = np.random.choice(image_size, noise_size)

    noise = np.random.choice([noised_image.min(), noised_image.max()], noise_size)
    noised_image.flat[random_indices] = noise

    return noised_image


src_image = cv2.imread(DATA_DIR / "kitty.png", cv2.IMREAD_GRAYSCALE) 

# add 10 % salt-and-pepper noise
noised_image = add_salt_and_pepper_noise(src_image)

dst_image = cv2.medianBlur(noised_image, ksize=7)

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 18))

ax1.imshow(noised_image, cmap="gray")
ax1.axis("off")

ax2.imshow(dst_image, cmap="gray")
ax2.axis("off")

plt.tight_layout()

##### **Дискретный базис Фурье**

In [ ]:
# размеры изображения
H = 128
W = 256

# реальная часть базиса Re(phi)
cos_phi = np.zeros((H, W))

# координаты в частотной области
u = 2
v = 1

for y in range(H):
    for x in range(W):
        cos_phi[y, x] = np.cos(2 * np.pi * (u * x / W + v * y / H))

plt.figure(figsize=(6, 6))
plt.imshow(cos_phi, cmap="gray", vmin=-1, vmax=1)
plt.plot(u, v, "o", color="red")
plt.xlabel("u")
plt.ylabel("v")
plt.title(f"u, v = {u}, {v}");


##### **Быстрое преобразование Фурье (FFT)**

In [ ]:
f = cv2.imread(DATA_DIR / "blown.png", cv2.IMREAD_GRAYSCALE)

# FFT
F = np.fft.fft2(f)

# спектр Фурье
abs_F = np.abs(F)

# фазовый спектр Фурье
arg_F = np.angle(F)

# Visualization
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,10))

ax1.imshow(f, cmap="gray")
ax1.set_xlabel("x")
ax1.set_ylabel("y")

ax2.imshow(np.log(1 + abs_F), cmap="gray")
ax2.set_xlabel("u")
ax2.set_ylabel("v")

ax3.imshow(arg_F, cmap="gray")
ax3.set_xlabel("u")
ax3.set_ylabel("v")

plt.tight_layout()

##### **Сдвиг низких частот в центр**

In [ ]:
f = cv2.imread(DATA_DIR / "blown.png", cv2.IMREAD_GRAYSCALE)

F = np.fft.fft2(f)

# сдвиг низких частот в центр
F = np.fft.fftshift(F)

# сдвиг обратную сторону
# F = np.fft.ifftshift(F)

abs_F = np.abs(F)
arg_F = np.angle(F)

# Visualization
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,10))

ax1.imshow(f, cmap="gray")
ax1.axis("off")

ax2.imshow(np.log(1 + abs_F), cmap="gray")
ax2.axis("off")

ax3.imshow(arg_F, cmap="gray")
ax3.axis("off")

plt.tight_layout()

##### **Обратное дискретное преобразование Фурье**

In [ ]:
f = cv2.imread(DATA_DIR / "blown.png", cv2.IMREAD_GRAYSCALE)

# преобразование Фурье
F = np.fft.fft2(f)

# обратное преобразование Фурье
g = np.fft.ifft2(F)

# вычисление реальной части комплексной матрицы изображения
g = g.real

# visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 18))

ax1.imshow(f, cmap="gray")
ax1.axis("off")

ax2.imshow(g, cmap="gray")
ax2.axis("off")

plt.tight_layout()

##### **Фильтрация изображений в частотной области**

##### *Идеальный фильтр низких частот*

In [ ]:
f = np.float32(
    cv2.imread(DATA_DIR / "pattern.png", cv2.IMREAD_GRAYSCALE)
)

H, W = f.shape
cx, cy = W // 2, H // 2

# сетка координат пикселей на изображении
u, v = np.meshgrid(np.arange(H), np.arange(W), indexing="ij")

# расстояние до центра изображения
D = np.sqrt((u - cx) ** 2 + (v - cy) ** 2)

# маска низких частот
M = np.where(D < H / 20, 1, 0)

# преобразование Фурье + сдвиг спектра
F = np.fft.fftshift(np.fft.fft2(f))

# преобразование Фурье разобьем на спектр и фазовый спектр
abs_F = np.abs(F)
arg_F = np.angle(F)

# фильтруем спектр
abs_F = abs_F * M

# спектр и фазовый спектр объединяем в преобразование Фурье
F = abs_F * np.exp(1j * arg_F)

# обратное преобразование Фурье
g = np.fft.ifft2(np.fft.ifftshift(F))
g = np.real(g)

# Visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(6, 6))

ax1.imshow(f, cmap="gray")
ax1.axis("off")

ax2.imshow(np.log(1 + np.abs(F)), cmap="gray")
ax2.axis("off")

ax3.imshow(M, cmap="gray")
ax3.axis("off")

ax4.imshow(g, cmap="gray")
ax4.axis("off")

plt.tight_layout()

##### *Идеальный фильтр высоких частот*

In [ ]:
# маска высоких частот
M = np.where(D > H / 20, 1, 0)

# преобразование Фурье + сдвиг спектра
F = np.fft.fftshift(np.fft.fft2(f))

# преобразование Фурье разобьем на спектр и фазовый спектр
abs_F = np.abs(F)
arg_F = np.angle(F)

# фильтруем спектр
abs_F = abs_F * M

# спектр и фазовый спектр объединяем в преобразование Фурье
F = abs_F * np.exp(1j * arg_F)

# обратное преобразование Фурье
g = np.fft.ifft2(np.fft.ifftshift(F))
g = np.real(g)

# Visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(6, 6))

ax1.imshow(f, cmap="gray")
ax1.axis("off")

ax2.imshow(np.log(1 + np.abs(F)), cmap="gray")
ax2.axis("off")

ax3.imshow(M, cmap="gray")
ax3.axis("off")

ax4.imshow(np.real(g), cmap="gray")
ax4.axis("off")

plt.tight_layout()

#### **Домашняя работа 2**

##### **Задача**
Реализуйте программу фильтрации в частотной области
шума на изображении колец Сатурна ``data/saturn_rings.png``.

##### **Теория**

- корреляция ядра и изображения
- свертка ядра и изображения
- сепарабельные свертки (как проверить сепарабельность, как найти  сепарабельное разложение ядра)
- box фильтр
- гауссовский фильтр
- медианный фильтр
- прямое и обратное преобразование Фурье (спектр, фазовый спектр свойства спектра при сдвиге и повороте изображения)
- фильтрация изображений в частотной области

